In [1]:
import os
os.environ['PYTENSOR_FLAGS'] = 'mode=FAST_COMPILE,optimizer=fast_compile,floatX=float32,cxx='

import numpy as np
import pymc as pm
import pytensor.tensor as pt

In this notebook, we demonstrate the initialization capability of PyMC's ADVI.

We will fit a simple 1d normal distribution using ADVI and compare the initializations.


Specifically, we will answer the following questions.

1. For unknown parameters in the likelihood, what does PyMC assign as the starting unconstrained mean $\tilde{\mu}$?
    - What is this value by default?
    - How is this value impacted by our input?

We will begin by looking at a Normal-Normal model and a beta-binomial model. In the normal case, no transformation is required so we expect that $\tilde{\mu}$ is equal to the input starting value. In the beta-binomial case, we know that PyMC uses a sigmoid transformation to move from $\mathbb{R}$ to $(0, 1)$, so we expect that $\tilde{\mu}$ is equal to the logit of the input starting value (since the starting value is input *in the constrained space*).

In [70]:
init_mean = 0.2
with pm.Model() as init_model:
    mu = pm.Normal("mu", 0, 1)
    likelihood = pm.Normal('y', mu=mu, sigma=1, observed=1)
    advi = pm.ADVI(start={'mu': init_mean})
    init_fit = advi.fit(0)

with pm.Model() as default_model:
    mu = pm.Normal("mu", 0, 1)
    likelihood = pm.Normal('y', mu=mu, sigma=1, observed=1)
    advi = pm.ADVI()
    default_fit = advi.fit(0)

# This is what \tilde{\mu} and \tilde{\sigma} are from the default and our input
print(f"Default initialization is: {default_fit.mean.eval()}, {default_fit.std.eval()}")
print(f"Manual initialization is: {init_fit.mean.eval()}, {init_fit.std.eval()}")

Output()

Initialization only


Output()

Initialization only


Default initialization is: [0.], [0.6931472]
Manual initialization is: [0.2], [0.6931472]


Now, let's use a Beta model, as an example of when a transformation is required, to determine whether manual initialization is changing the constrained value in $(0, 1)$ or unconstrained mean in $\mathbb{R}$.

When we give PyMC a starting value $x$ for a parameter $\theta$ where PyMC uses transformation $T$ to transform from $\mathbb{R}$ to the support of $\theta$, PyMC sets the unconstrained mean $\tilde{\mu}$ (of the underlying normal distribution used in ADVI) to $T^{-1}(x)$.

To examine this functionality, we will sample from the posterior and take the empirical mean and std of posterior samples to compare to the starting value choice for $\theta$ of $0.2$.

In [ ]:
init_mean = 0.2
with pm.Model() as beta_model:
    # Define priors
    theta = pm.Beta("theta", 2, 5)
    likelihood = pm.Binomial('obs', n=10, p=theta, observed=2)
    advi = pm.ADVI(start={'theta': init_mean})
    approx = advi.fit(0)

# check if mu = approx.params_dict.get("mu") is equal to logit(init_mean)
mu = approx.params_dict.get("mu")
logit_mu = np.log(init_mean / (1 - init_mean))
print(logit_mu)
print(mu.eval())

Output()

Initialization only


-1.3862943611198906
[-1.3862944]
[theta]


Now, we'd like to understand what is happening with a possibly more contrived example.

Again, we'd like to understand what PyMC sets as $\tilde{\mu}$ by default and when we manually change the starting value.

When we put 'None' as the observed= values of the likelihood distribution, PyMC treats it as a free RV and we may directly access the underlying starting values for the unknown parameters in the likelihood.

First, we look at the case where $X$~$\mathcal{N}(1, 3) * 2$ and $Y$~$\mathcal{N}(X, 1)$.

Note that we use pm.Deterministic so that everything plays nice in PyMC's code.

In [75]:
with pm.Model() as weird_model:
    # Define priors
    x_sub = pm.Normal("x_sub", 2, 3)
    x = pm.Deterministic("x", x_sub * 2)
    likelihood = pm.Normal('y', mu=x, sigma=1, observed=None)
    advi = pm.ADVI()
    approx = advi.fit(0)

mu = approx.params_dict.get("mu")
print(mu.eval())
# this returns the initial value of the tilde{mu} of x_sub and the initial value of the tilde{mu} of y

Output()

Initialization only


[2. 4.]


In [78]:
with pm.Model() as weird_model:
    # Define priors
    x_sub = pm.Normal("x_sub", 2, 3)
    x = pm.Deterministic("x", x_sub * 2)
    likelihood = pm.Normal('y', mu=x, sigma=1, observed=None)
    advi = pm.ADVI(start={'x_sub': 10}) # we may change either x_sub or x here but only x_sub actually changes the start point of y!
    approx = advi.fit(0)

mu = approx.params_dict.get("mu")
print(mu.eval())
# this returns the initial value of the tilde{mu} of x_sub and the initial value of the tilde{mu} of y

Output()

Initialization only


[10. 20.]


In [77]:
with pm.Model() as weird_model:
    # Define priors
    x_sub = pm.Normal("x_sub", 2, 3)
    x = pm.Deterministic("x", x_sub * 2)
    likelihood = pm.Normal('y', mu=x, sigma=1, observed=None)
    advi = pm.ADVI(start={'x': 10}) # we may change either x_sub or x here!
    approx = advi.fit(0)

mu = approx.params_dict.get("mu")
print(mu.eval())
# this returns the initial value of the tilde{mu} of x_sub and the initial value of the tilde{mu} of y

Output()

Initialization only


[2. 4.]


Now let's look at another weird prior:
$X$~Uniform$(-3, 3) + 10$ and $Y$~$\mathcal{N}(X, 1)$.

In [80]:
with pm.Model() as weird_model:
    # Define priors
    uni = pm.Uniform("uni_sub", -3, 3)
    uni_2 = pm.Deterministic("uni_2", uni + 10)
    likelihood = pm.Normal('y', mu=uni_2, sigma=1, observed=None)
    advi = pm.ADVI()
    approx = advi.fit(0)

mu = approx.params_dict.get("mu")
print(mu.eval())

Output()

Initialization only


[ 0. 10.]


In [82]:
with pm.Model() as weird_model:
    # Define priors
    uni = pm.Uniform("uni_sub", -3, 3)
    uni_2 = pm.Deterministic("uni_2", uni + 10)
    likelihood = pm.Normal('y', mu=uni_2, sigma=1, observed=None)
    advi = pm.ADVI(start={'uni_2': 10.5})  # again, only changing uni_sub actually changes the initial values
    approx = advi.fit(0)

mu = approx.params_dict.get("mu")
print(mu.eval())

Output()

Initialization only


[ 0. 10.]


In [83]:
with pm.Model() as weird_model:
    # Define priors
    uni = pm.Uniform("uni_sub", -3, 3)
    uni_2 = pm.Deterministic("uni_2", uni + 10)
    likelihood = pm.Normal('y', mu=uni_2, sigma=1, observed=None)
    advi = pm.ADVI(start={'uni_sub': 0.5})  # again, only changing uni_sub actually changes the initial values
    approx = advi.fit(0)

mu = approx.params_dict.get("mu")
print(mu.eval())

Output()

Initialization only


[ 0.33647215 10.5       ]


In [84]:
with pm.Model() as weird_model:
    # Define priors
    uni = pm.Uniform("uni_sub", -3, 3)
    uni_2 = pm.Deterministic("uni_2", uni + 10)
    likelihood = pm.Normal('y', mu=uni_2, sigma=1, observed=None)
    advi = pm.ADVI(start={'uni_sub': 10.5})  # can we initialize out of support?
    approx = advi.fit(0)

mu = approx.params_dict.get("mu")
print(mu.eval())

/Users/madelynandersen/miniconda3/envs/pymc_env_pm/lib/python3.11/site-packages/pytensor/tensor/elemwise.py:710: RuntimeWarning: invalid value encountered in log
  variables = ufunc(*ufunc_args, **ufunc_kwargs)


Output()

Initialization only


[nan nan]


# Appendix

Let's look at what samples from the posterior give us empirically for the constrained mean and std of the posterior of $\theta$.

In [21]:

idata = approx.sample(1_000_000, random_seed=0)
theta_samps = np.asarray(idata.posterior["theta"]).ravel()
print("Posterior samples (constrained theta):")
print("  mean =", theta_samps.mean(), "std =", theta_samps.std(ddof=1))

# ADVI mean-field uses 'mu' and 'rho' (rho -> sigma via softplus) in *unconstrained space*.
mu = approx.params_dict.get("mu")
rho = approx.params_dict.get("rho")

mu_z = mu[0].eval()
sigma_z = float(np.log1p(np.exp(rho[0].eval())))  # softplus(rho)

print("\nUnconstrained (R) params for theta after initialization:")
print("  mu_z    =", mu_z)
print("  sigma_z =", sigma_z)

# Let's take samples in unconstrained space and transform them to constrained space to verify the empirical mean/std
# sample from N(mu_z, sigma_z^2)
from scipy.stats import norm
unconstrained_samples = norm.rvs(loc=mu_z, scale=sigma_z, size=1_000_000, random_state=0)
# transform that to (0, 1) via sigmoid transformation
mc_samples = 1.0 / (1.0 + np.exp(-unconstrained_samples))
print("\nMC from z-space params (should match approx.sample mean):")
print("  mean =", mc_samples.mean(), "std =", mc_samples.std(ddof=1))


Posterior samples (constrained theta):
  mean = 0.2204934 std = 0.1132369

Unconstrained (R) params for theta after initialization:
  mu_z    = -1.3862944
  sigma_z = 0.6931471824645996

MC from z-space params (should match approx.sample mean):
  mean = 0.2208659595507493 std = 0.11344936075158488


Since the posterior samples and MC samples have matching summary statistics, we can see that measure-transport like we just performed is what is happening under the hood when the "constrained mean" is initialized.